## This code will:
#### - Generate a new svg with classes and parent-id attribute using the 'context' defined in group_names
#### - Modified the ngv_portal_metab_model_v1_processed.json to change a bit the format of the file produced by Polina

## Before run it:
#### 1) Open the [original](https://bbpteam.epfl.ch/project/issues/browse/NGVDISS-351?focusedCommentId=169995&page=com.atlassian.jira.plugin.system.issuetabpanels:comment-tabpanel#comment-169995) svg in draw.io.
#### 2) Activate the 'plugins/svgdata.js' in Extra -> Plugins so an ID is added to each element
#### 3) Export it as XML file. It has the information about ID and parent group.
#### 3) Save the svg so the pluging adds the ids


In [ ]:
XML_FILE_PATH = './model_components_svg_v4.drawio.xml'
SVG_FILE_PATH = './model_components_svg_v4.svg'
SVG_OUTPUT_FILE_PATH = './model_components_svg_v4_processed.svg'

In [ ]:
from lxml import etree
import json
import re

# Fetch all child nodes and parents

In [ ]:
root = etree.parse(XML_FILE_PATH).getroot()
# get all items
items = root.findall('.//mxCell')
len(items)

In [ ]:
i = items[544]
i.get('parent')

In [ ]:
# create map from element_id and parent 'context'
parent_mapping = {}
full_parent_list = []
parent_child_mapping = {} # to test

for item in items:
  parent_id = item.get('parent')
  if parent_id is None or parent_id in ['0', '1']:
    continue # skip also the elements with no group

  child_id = item.get('id')

  if parent_id in parent_child_mapping:
    parent_child_mapping[parent_id].append(item.get('value'))
  else:
    parent_child_mapping[parent_id] = [item.get('value')]
  
  parent_mapping[child_id] = parent_id
  full_parent_list.append(parent_id) if parent_id not in full_parent_list else full_parent_list
  
print(full_parent_list)

In [ ]:
# parent_mapping
# parent_child_mapping

### assign names to the 'contexts' above

if the SVG change, then these ids will change

to generate it pick element in the svg and check the property id. in the xml check the parent.

In [ ]:
group_names = {
  '619': 'blood-vessel',
  '623': 'neuron-cytosol', 
  'd4GJFtNBkq4fSjHgL4VR-654': 'neuron-mitochondrion',
  '518': 'astrocyte-cytosol',
  '567': 'astrocyte-mitochondrion',
}

In [ ]:
# with open('./parent_mapping.json', 'w') as f:
#   f.write(json.dumps(parent_mapping))

# 1) Add classes to SVG

In [ ]:
root_svg = etree.parse(SVG_FILE_PATH).getroot()

### cleanup pointer events

In [ ]:
# delete pointer event as attribute
counter = 0
for elem in root_svg.xpath(".//*[@pointer-events]"):
  if 'pointer-events' not in elem.attrib: continue
  counter += 1
  del elem.attrib['pointer-events']
print(counter)

In [ ]:
# delete pointer event in element style
counter = 0
for elem in root_svg.xpath(".//*[contains(@style,'pointer-events')]"):
  new_style = elem.get('style').replace('pointer-events: none;', '').replace('pointer-events: all;', '')
  elem.set('style', new_style)
  counter += 1
print(counter)

In [ ]:
# add none to root svg to it let the internal elements to be clicked
svg_style = root_svg.get('style')
new_style = svg_style + 'pointer-events: none;'
root_svg.set('style', new_style)

### add class to clickable elements

In [ ]:
pointer_style = 'pointer-events: all;'

def add_pointer_style(element):
  elem_style = element.get('style')
  new_elem_style = None
  if elem_style is None:
    new_elem_style = pointer_style
  else:
    if elem_style.endswith(';'):
      new_elem_style = elem_style + pointer_style
    else:
      new_elem_style = elem_style + ';' + pointer_style

  element.set('style', new_elem_style)


def bubble_up_till_id(element):
  if element is None:
    return None
  id = element.get('id')
  if id is None:
    return bubble_up_till_id(element.getparent())
  return element

In [ ]:
clickable_counter = 0
not_clickable_counter = 0

elem_query_str = './/{http://www.w3.org/1999/xhtml}font'

found_elements = root_svg.findall(elem_query_str)

clickable_elements_list = []

In [ ]:
for element in found_elements:
  element_g = bubble_up_till_id(element)
  if element_g is None:
    continue

  cell_id = element_g.get('id')
  elem_id = cell_id.replace('cell-', '')

  # process only the element which parent is in defined contexts
  parent_id = parent_mapping[elem_id] if elem_id in parent_mapping else None
  if parent_id is None:
    not_clickable_counter += 1
    continue

  element.set('class', 'clickable')
  element.set('parent-id', group_names[parent_id])
  add_pointer_style(element)
  clickable_counter += 1
  clickable_elements_list.append(element)

print(f'Clickable: {clickable_counter} / {clickable_counter + not_clickable_counter}')

### remove unused 'content' attribute on svg
the content is only used by draw.io and as we have the original image this is not necessary after processing

In [ ]:
del root_svg.attrib['content']

### fix issue with 'vasculature dynamics' that is one element but drawio divided in two.

In [ ]:
output_str = etree.tostring(root_svg, pretty_print=False).decode('utf-8')

In [ ]:
output_str_fixed = re.sub(r'(?<=>vasculature).+?>(?=dynamics<)', ' ', output_str)

### save to file
(on-click not working when importing)

In [ ]:
with open(SVG_OUTPUT_FILE_PATH, 'w') as f:
  f.write(output_str_fixed)

### (for debugging) copy svg to index.html (to test)

In [ ]:
with open('index.html', 'r') as f:
  html = f.read()

import re
# content = output_str_fixed
content = etree.tostring(root_svg, pretty_print=False).decode('utf-8').replace('\n', '')

new_html = re.sub(
  r'<span class="svg-container">(.*)</span>',
  f'<span class="svg-container">{content}</span>',
#   f'<span class="svg-container"></span>',
  html,
)

with open('index.html', 'w') as f:
  f.write(new_html)

---

# 2) Generate list for intemediate mapping

We need to map the svg click to the actual [information dictionary](https://bbpgitlab.epfl.ch/molsys/metabolismndam/-/blob/main/sim/metabolism_unit_models/FINAL_CLEAN/data_model_full/ngv_portal_metab_model_v1.json). To do that I generated an array with all the 'innerText--parent' and Polina mapped those with the actual entry on the information dict linked above.

What I created
```
[
  "Thiolase--neuron-mitochondrion",
  "SDH--neuron-mitochondrion",
  ...
]
```
What Polina [produced](https://bbpgitlab.epfl.ch/molsys/metabolismndam/-/blob/main/sim/metabolism_unit_models/FINAL_CLEAN/data_model_full/ngvportal_imgpicker_mapping_dict.json)
```
{
  "Thiolase--neuron-mitochondrion": ["Ketones_thiolase"],
  ...
}
```

In [ ]:
intermediate_mapping_list = []
for clickable_elem in clickable_elements_list:
  parent_id = clickable_elem.get('parent-id')
  text = clickable_elem.text
  context_name = parent_id
  intermediate_mapping_list.append(f'{text}--{context_name}')

In [ ]:
intermediate_mapping_list[:5]

There is a small issue with `Na+,K+-ATPase` that the text that lxml fetches is only `Na`

In [ ]:
match_indexes = []
for index, item in enumerate(intermediate_mapping_list):
  if item.startswith('Na--'):
    match_indexes.append(index)
    print(item)

for match_index in match_indexes:
  intermediate_mapping_list[match_index] = intermediate_mapping_list[match_index].replace('Na--', 'Na+,K+-ATPase--')
print('elements were modified')

In [ ]:
with open('intermediate_mapping_list.json', 'w') as f:
  f.write(json.dumps(intermediate_mapping_list, indent=2))

---

# 3) Transform a bit the ngv_portal_metab_model_v*.json 
- converting array of dicts to dict `[ { "MCT1_LAC_a_J": {} }, {} ]` to `{ "MCT1_LAC_a_J": {}, "" }`
- remove gpfs path

In [ ]:
model_full_info_file = './ngv_portal_metab_model_v2.json'

In [ ]:
with open(model_full_info_file, 'r') as f:
  enzymes_info = json.loads(f.read())

In [ ]:
def extract_plot_paths(value):
  plots = []
  plots.append(value['figure_combo'])
  for variable in value['variables']:
    for fig in variable['figure']:
      plots.append(fig)
  return plots

In [ ]:
new_enzymes_info = {}
plots_dict = {}

for item in enzymes_info:
  key = list(item.keys())[0]
  value = list(item.values())[0]
  plots_dict[key] = extract_plot_paths(value)
  new_enzymes_info[key] = value

In [ ]:
# new_enzymes_info
# new_enzymes_info['MCT1_LAC_a_J']
plots_dict['MCT1_LAC_a_J']

In [ ]:
gpfs_path_to_remove = '/gpfs/bbp.cscs.ch/project/proj34/metabolismndam/sim/metabolism_unit_models/optimiz_unit/'
new_enzymes_info_str = json.dumps(new_enzymes_info)
new_enzymes_info_str = new_enzymes_info_str.replace(gpfs_path_to_remove, '')

In [ ]:
with open(model_full_info_file.replace('.json', '_processed.json'), 'w') as f:
  f.write(new_enzymes_info_str)

with open('plots_dict.json', 'w') as f:
  f.write(json.dumps(plots_dict))

In [ ]:
!open .

---

# 4) Sync plots from gpfs to openshift static content

Mount gpfs

`sshfs -o reconnect -o defer_permissions bbpv1.epfl.ch:/gpfs ~/gpfs`

Log in openshift

`oc login https://ocp.bbp.epfl.ch:8443`

Get the static data pod

`oc get pods | grep ngv-portal-static-data`

Copy the pod name in the next command

```
oc rsync \
~/gpfs/bbp.cscs.ch/project/proj34/metabolismndam/sim/metabolism_unit_models/optimiz_unit/enzymes/figures \
ngv-portal-static-data-9-bkrq7:/usr/share/nginx/html/data/metabolism/digital-reconstruction/enzymes
```